
# 🕳️ Dark Pool vs Lit Venue Execution Simulator

_Date generated: 2025-09-03_

Simulate order execution across **lit** and **dark** venues with a simple **price/flow microstructure** model:
- Midprice process with **alpha/toxicity** bursts
- Venues: **Lit** (quoted spread, queue/impact) and **Dark** (midpoint crosses, random contra liquidity, information risk)
- Strategies: **Lit-only**, **Dark-only (midpoint peg)**, **Heuristic SOR**, **Bandit SOR (UCB)**
- Metrics: **implementation shortfall**, **fill rate**, **adverse selection**, **venue share**, **cost distribution**

> Plug this into your execution stack to stress-test routing logic and market regimes.


## 0) Parameters

In [ ]:

# Parent order
SIDE = "BUY"          # BUY or SELL
Q_PARENT = 150_000    # total shares
SLICES = 60           # child order slices

# Market process
P0 = 100.0
DT = 1/78
SIGMA_D = 0.012       # daily vol
ALPHA_PROB = 0.15     # prob of informed/toxic burst per slice
ALPHA_MAG  = 0.0025   # expected drift during toxic burst (per slice)

# Lit venue
SPREAD_BPS = 6        # quoted half-spread (bps) -> total spread = 2*half
QUEUE_COST_BPS = 1.0  # slippage/queue cost per slice when we cross
MARKET_IMPACT_C = 2e-6 # linear impact per share

# Dark venue
DARK_CROSS_RATE = 0.35 # baseline chance of finding contra per slice
DARK_SIZE_MEAN  = 4_000
DARK_SIZE_SD    = 2_000
DARK_TOXIC_SLIP = 0.0008  # extra adverse move after dark fill if flow is toxic

# Bandit SOR
UCB_C = 1.2

# Random seed
SEED = 7


## 1) Setup & Helpers

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(SEED)

def bps(x): return x/1e4
sign = +1 if SIDE.upper()=="BUY" else -1

def drawdown_curve(x: pd.Series) -> pd.Series:
    return x / x.cummax() - 1.0


## 2) Market & Toxicity Process

In [ ]:

def simulate_paths(T=SLICES, p0=P0, sigma_d=SIGMA_D, dt=DT, alpha_prob=ALPHA_PROB, alpha_mag=ALPHA_MAG):
    sigma = sigma_d * np.sqrt(dt)
    mid = np.zeros(T+1); mid[0] = p0
    tox = np.zeros(T)    # 1 if informed burst this slice
    drift = np.zeros(T)
    for t in range(T):
        tox[t] = np.random.rand() < alpha_prob
        drift[t] = (alpha_mag * (1 if np.random.rand()>0.5 else -1)) if tox[t] else 0.0
        ret = drift[t] + np.random.normal(0, sigma)
        mid[t+1] = max(0.01, mid[t] * (1 + ret))
    return pd.Series(mid, name="mid"), pd.Series(tox.astype(int), index=range(T), name="toxic"), pd.Series(drift, index=range(T), name="drift")

mid, toxic, drift = simulate_paths()
mid.head(), toxic.value_counts()


## 3) Venue Execution Models

In [ ]:

def exec_lit(q_req, mid_px, side=SIDE, spread_bps=SPREAD_BPS, queue_bps=QUEUE_COST_BPS, impact_c=MARKET_IMPACT_C):
    # Cross the spread and pay queue/impact proportional to size
    half_spread = bps(spread_bps) * mid_px
    # execution price on lit (cross): buy at ask, sell at bid
    if side.upper()=="BUY":
        px = mid_px + half_spread + bps(queue_bps)*mid_px + impact_c*q_req
    else:
        px = mid_px - half_spread - bps(queue_bps)*mid_px - impact_c*q_req
    return q_req, px

def exec_dark(q_req, mid_px, tox_flag, side=SIDE, cross_rate=DARK_CROSS_RATE, size_mean=DARK_SIZE_MEAN, size_sd=DARK_SIZE_SD, tox_slip=DARK_TOXIC_SLIP):
    # Random crossing size if contra available
    if np.random.rand() > cross_rate:
        return 0.0, None, 0.0  # no fill
    size = max(0.0, np.random.normal(size_mean, size_sd))
    q_fill = min(q_req, size)
    # midpoint execution
    px = mid_px
    # Adverse selection kick if flow toxic (mid moves against us post-trade)
    post_move = (tox_slip * mid_px) * (+1 if side.upper()=="BUY" else -1) if tox_flag else 0.0
    # Return fill and realized post-move penalty (notional impact metric)
    return q_fill, px, post_move


## 4) Routing Strategies

In [ ]:

def alloc_even(q_slice, dark_weight=0.5):
    q_dark = q_slice * dark_weight
    q_lit = q_slice - q_dark
    return q_dark, q_lit

def strategy_lit_only(q_slice, ctx):
    return 0.0, q_slice

def strategy_dark_only(q_slice, ctx):
    return q_slice, 0.0

def strategy_sor_heuristic(q_slice, ctx):
    # Heuristic: if toxic -> go lit (avoid dark adverse), else try dark
    dw = 0.2 if ctx['toxic'] else 0.7
    return alloc_even(q_slice, dark_weight=dw)

class UCBRouter:
    def __init__(self, c=1.0):
        self.c = c
        self.n = np.zeros(2)  # 0=dark, 1=lit
        self.val = np.zeros(2)
        self.t = 0

    def select(self):
        self.t += 1
        ucb = np.zeros(2)
        for i in range(2):
            if self.n[i] == 0:
                ucb[i] = np.inf
            else:
                ucb[i] = self.val[i] + self.c * np.sqrt(np.log(self.t)/self.n[i])
        return int(np.argmax(ucb))

    def update(self, arm, reward):
        # reward should be *negative cost* (higher is better)
        self.n[arm] += 1
        self.val[arm] += (reward - self.val[arm]) / self.n[arm]

def strategy_bandit_ucb(q_slice, ctx):
    router: UCBRouter = ctx['router']
    # choose arm each slice; map to allocation
    arm = router.select()   # 0=dark, 1=lit
    if arm == 0:
        return q_slice, 0.0
    else:
        return 0.0, q_slice


## 5) Execution Loop

In [ ]:

def run_strategy(strategy_fn, label, q_parent=Q_PARENT, slices=SLICES, side=SIDE):
    q_rem = q_parent
    fills = []
    router = UCBRouter(c=UCB_C) if label=="SOR-UCB" else None

    for t in range(slices):
        if q_rem <= 0: break
        # VWAP-y slice sizing (equal time slicing here)
        q_slice = q_parent / slices
        q_slice = min(q_slice, q_rem)

        ctx = {"toxic": bool(toxic.iloc[t]), "router": router}
        q_dark_req, q_lit_req = strategy_fn(q_slice, ctx)

        mid_t = mid.iloc[t]
        # Dark venue
        qd, dark_px, post_move = exec_dark(q_dark_req, mid_t, tox_flag=ctx["toxic"], side=side)
        # Lit venue (execute whatever isn't filled in dark)
        ql_req = q_lit_req + (q_dark_req - qd)
        ql, lit_px = (0.0, None)
        if ql_req > 0:
            ql, lit_px = exec_lit(ql_req, mid_t, side=side)

        # Realized cost per fill vs arrival mid P0
        cost_dark = 0.0
        if qd > 0:
            # buy at midpoint -> opportunity cost = (px - P0) * qd (sign-adjusted)
            if side.upper()=="BUY":
                cost_dark = (dark_px - P0) * qd # type: ignore
            else:
                cost_dark = (P0 - dark_px) * qd # type: ignore
            # add post trade adverse selection penalty (notionally)
            cost_dark += abs(post_move) * qd

        cost_lit = 0.0
        if ql > 0:
            if side.upper()=="BUY":
                cost_lit = (lit_px - P0) * ql # type: ignore
            else:
                cost_lit = (P0 - lit_px) * ql # type: ignore

        # Update bandit rewards (negative cost -> reward)
        if router is not None:
            if qd > 0:
                router.update(0, reward=-cost_dark/max(q_slice,1))
            if ql > 0:
                router.update(1, reward=-cost_lit/max(q_slice,1))

        fills.append({
            "t": t, "mid": mid_t, "toxic": int(ctx["toxic"]),
            "q_dark_req": q_dark_req, "q_dark_fill": qd, "px_dark": dark_px,
            "q_lit": ql, "px_lit": lit_px,
            "cost_dark": cost_dark, "cost_lit": cost_lit
        })
        q_rem -= (qd + ql)

    df = pd.DataFrame(fills)
    if df.empty:
        return label, df, {"is": np.nan}
    # Metrics
    total_fill = df["q_dark_fill"].sum() + df["q_lit"].sum()
    is_cost = df["cost_dark"].sum() + df["cost_lit"].sum()
    avg_px = ( (df["q_dark_fill"]*df["px_dark"].fillna(0)).sum() + (df["q_lit"]*df["px_lit"].fillna(0)).sum() ) / max(total_fill,1)
    # Adverse selection proxy: signed mid move after dark fills (we used fixed add-on; here we report fraction of toxic slices that filled)
    dark_fill_toxic = df.loc[df["q_dark_fill"]>0, "toxic"].mean() if (df["q_dark_fill"]>0).any() else np.nan
    fill_rate = total_fill / q_parent
    venue_share_dark = df["q_dark_fill"].sum() / max(total_fill,1)
    metrics = {
        "label": label,
        "fill_rate": fill_rate,
        "venue_share_dark": venue_share_dark,
        "avg_px": avg_px,
        "IS_cost_$": is_cost,
        "dark_fill_toxic_rate": dark_fill_toxic
    }
    return label, df, metrics

strategies = {
    "Lit-only": strategy_lit_only,
    "Dark-only": strategy_dark_only,
    "SOR-Heuristic": strategy_sor_heuristic,
    "SOR-UCB": strategy_bandit_ucb
}

runs = []
for name, fn in strategies.items():
    label, df, met = run_strategy(fn, name)
    runs.append((label, df, met))

metrics = pd.DataFrame([m for _,_,m in runs]).set_index("label")
metrics


## 6) Plots

In [ ]:

# Cost breakdown bars
plt.figure(figsize=(10,3.5))
plt.bar(metrics.index, metrics["IS_cost_$"])
plt.title("Implementation Shortfall Cost by Strategy ($)")
plt.tight_layout(); plt.show()

# Venue shares & fill rates
fig, ax = plt.subplots(1,2, figsize=(11,3.5))
ax[0].bar(metrics.index, metrics["venue_share_dark"])
ax[0].set_title("Dark Venue Share")
ax[1].bar(metrics.index, metrics["fill_rate"])
ax[1].set_title("Fill Rate")
plt.tight_layout(); plt.show()

# Toxicity timeline & mid
plt.figure(figsize=(10,3))
plt.plot(mid.index[:-1], mid.values[:-1], label="Mid") # type: ignore
plt.scatter(mid.index[:-1], mid.values[:-1], c=["red" if t else "green" for t in toxic.values], s=10, label="Toxic slice") # type: ignore
plt.title("Midprice Path with Toxic Slices")
plt.legend(); plt.tight_layout(); plt.show()

# Distribution of per-slice costs for best strategy
best = metrics["IS_cost_$"].idxmin()
df_best = [df for lbl,df,_ in runs if lbl==best][0]
per_slice_cost = df_best["cost_dark"].fillna(0) + df_best["cost_lit"].fillna(0)
plt.figure(figsize=(10,3))
per_slice_cost.hist(bins=30, edgecolor="black")
plt.title(f"Per-slice Cost Distribution — {best}")
plt.tight_layout(); plt.show()


## 7) What‑If Scenarios

In [ ]:

def run_sensitivity(**kwargs):
    # override globals for quick sweeps
    globals().update(kwargs)
    global mid, toxic, drift
    mid, toxic, drift = simulate_paths(alpha_prob=ALPHA_PROB, alpha_mag=ALPHA_MAG)
    # re-run
    out = {}
    for name, fn in strategies.items():
        _, _, met = run_strategy(fn, name)
        out[name] = met
    return pd.DataFrame(out).T

sens_toxic = run_sensitivity(ALPHA_PROB=0.30)
sens_toxic[["IS_cost_$","venue_share_dark","fill_rate"]]
